### Importamos las librerias necesarias para comenzar el proceso de ETL 

In [18]:
import pandas as pd
import ast
import json
import numpy as np
from ast import literal_eval
import matplotlib.pyplot as plt
import seaborn as sns
from textblob import TextBlob
import herramientas
import warnings
warnings.filterwarnings("ignore")

***dataset: output_steam_games***

#### Extraccion de datos y primera exploracion

Se extraen los datos desde el archivo 'output_steam_games.json', se convierte en Dataframe y se observa su contenido.

In [19]:
#Leer el archivo línea por línea y cargar cada línea como un objeto JSON
with open('output_steam_games.json', 'r', encoding='utf-8') as archivo:
    data = [json.loads(line) for line in archivo]

#Convertir la lista de objetos JSON en un DataFrame
df_games = pd.DataFrame(data)
df_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


Se revisan los tipos de datos por columna y la cantidad de nulos.

In [20]:
herramientas.verifica_tipo_y_nulos(df_games)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,publisher,"[<class 'float'>, <class 'str'>]",20.00,80.00,96362
1,genres,"[<class 'float'>, <class 'list'>]",23.95,76.05,91593
2,app_name,"[<class 'float'>, <class 'str'>]",26.68,73.32,88312
3,title,"[<class 'float'>, <class 'str'>]",24.98,75.02,90360
4,url,"[<class 'float'>, <class 'str'>]",26.68,73.32,88310
5,release_date,"[<class 'float'>, <class 'str'>]",24.96,75.04,90377
6,tags,"[<class 'float'>, <class 'list'>]",26.54,73.46,88473
7,reviews_url,"[<class 'float'>, <class 'str'>]",26.68,73.32,88312
8,specs,"[<class 'float'>, <class 'list'>]",26.12,73.88,88980
9,price,"[<class 'float'>, <class 'str'>]",25.54,74.46,89687


Se observan filas que contienen vacíos en todo el registro. Se eliminan las mismas

In [21]:
df_games = df_games.dropna(how='all').reset_index(drop=True)
df_games.shape

(32135, 13)

Este conjunto contiene 13 columnas y quedó con 32135 filas luego de borrar las filas de NaN, aún contienen nulos en algunas columnas. Las columnas que contiene son:

- **publisher**: la empresa publicadora del contenido.
- **genres**: el género del item, es decir, del juego. Esta formado por una lista de uno o mas géneros por registro.
- **app_name**: el nombre del item, es decir, del juego.
- **title**:  el título del item.
- **url**:  la url del juego.
- **release_date**: la fecha de lanzamiento del item en formato 2018-01-04.
- **tags**:  la etiqueta del contenido. Esta formado por una lista de uno o mas etiquetas por registro.
- **reviews_url**:  la url donde se encuentra el review de ese juego.
- **specs**:  especificaciones de cada item. Es una lista con uno o mas string con las especificaciones.
- **price**:  el precio del item.
- **early_access**: indica el acceso temprano con un True/False.
- **id**:  el identificador único del contenido.
- **developer**: el desarrollador del contenido.

Se analizan si hay duplicados teniendo en cuenta la columna del id del item.

In [22]:
herramientas.verifica_duplicados_por_columna(df_games,'id')

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
13894,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games
14573,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games
74,NaN,NaN,NaN,NaN,http://store.steampowered.com/,NaN,NaN,NaN,NaN,19.99,False,NaN,NaN
30961,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)"


Se observa que solo hay dos items duplicados. Por un lado el 'id' 612880 que se observa es el mismo registro en los dos casos. Por lo que se puede eliminar cualquier de ellos. Por otra parte, se observa que la segunda dupla de duplicados el 'id' es NaN y la primera aparición presenta mas cantidad de NaN que la segunda aparición.

Primero se busca por el 'developer' si este juego está ya registrado.

In [23]:
df_games[df_games['developer']=='Rocksteady Studios,Feral Interactive (Mac)']

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
1068,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260/Batma...,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",http://steamcommunity.com/app/200260/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,200260,"Rocksteady Studios,Feral Interactive (Mac)"
30961,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham City - Game of the Year Edition,Batman: Arkham City - Game of the Year Edition,http://store.steampowered.com/app/200260,2012-09-07,"[Action, Open World, Batman, Adventure, Stealt...",NaN,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,NaN,"Rocksteady Studios,Feral Interactive (Mac)"
31617,"Warner Bros. Interactive Entertainment, Feral ...","[Action, Adventure]",Batman: Arkham Asylum Game of the Year Edition,Batman: Arkham Asylum Game of the Year Edition,http://store.steampowered.com/app/35140/Batman...,2010-03-26,"[Action, Batman, Stealth, Adventure, Third Per...",http://steamcommunity.com/app/35140/reviews/?b...,"[Single-player, Steam Achievements, Steam Trad...",19.99,False,35140,"Rocksteady Studios,Feral Interactive (Mac)"


Se puede ver que el item que no tiene 'id' es el 'id' 200260 y que el registro se encuentra completo. Por lo que se pueden borrar ambas filas dupliacadas.

In [24]:
# se eliminan los index deseados
filas_a_eliminar = [14573, 74, 30961]
df_games = df_games.drop(filas_a_eliminar)
# se verifica si hay dupliados
herramientas.verifica_duplicados_por_columna(df_games, 'id')

'No hay duplicados'

#### Modificacion del tipo de dato de ser necesario

- ***'release_date'***

Se necesita extraer el año de lanzamiento del item, para ello se hace una nueva columna con el dato si existe o con un "Dato no disponible" si no esta la fecha. Luego se elimina la columa 'release_date'.

In [25]:
# Se observan las cantidades de registros por cada fecha
df_games['release_date'].value_counts()

release_date
2012-10-16    100
2017-08-31     92
2017-09-26     89
2017-06-21     82
2017-07-25     78
             ... 
1988-04-16      1
2013-08-24      1
2011-05-07      1
2010-08-21      1
2018-10-01      1
Name: count, Length: 3582, dtype: int64

In [26]:
# Crea columna nueva con el año
df_games['release_anio'] = df_games['release_date'].apply(herramientas.extrae_anio)
# elimina la columna 'release_date'
df_games = df_games.drop('release_date', axis=1)
df_games.head()

,publisher,genres,app_name,title,url,tags,reviews_url,specs,price,early_access,id,developer,release_anio
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,2018
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,2018
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,2017
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,2017
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN,Dato no disponible


In [27]:
df_games['release_anio'].unique()

array(['2018', '2017', 'Dato no disponible', '1997', '1998', '2016',
       '2006', '2005', '2003', '2007', '2002', '2000', '1995', '1996',
       '1994', '2001', '1993', '2004', '1999', '2008', '2009', '1992',
       '1989', '2010', '2011', '2013', '2012', '2014', '1983', '1984',
       '2015', '1990', '1988', '1991', '1985', '1982', '1987', '1981',
       '1986', '2021', '2019', '1975', '1970', '1980'], dtype=object)

- ***'publisher'***
- ***'app_name'***
- ***'title'*** 
- ***'developer'***

En estas columnas se observan valores nulos, por lo que se decide completarlos con valores del tipo 'Dato no disponible' para que el tipo de dato sea uno solo en la columna.

In [28]:
# SColumnas a transformar
columnas_a_completar = ['publisher', 'app_name', 'title', 'developer']
# Se rellenan los nulos
df_relleno = df_games[columnas_a_completar].fillna('Sin dato disponible')
# Se borran las columnas originales y se concatenan las rellenas con todo el dataframe
df_games = pd.concat([df_games.drop(columnas_a_completar, axis=1), df_relleno], axis=1)
df_games.head()

,genres,url,tags,reviews_url,specs,price,early_access,id,release_anio,publisher,app_name,title,developer
0,"[Action, Casual, Indie, Simulation, Strategy]",http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
1,"[Free to Play, Indie, RPG, Strategy]",http://store.steampowered.com/app/643980/Ironb...,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,2018,"Making Fun, Inc.",Ironbound,Ironbound,Secret Level SRL
2,"[Casual, Free to Play, Indie, Simulation, Sports]",http://store.steampowered.com/app/670290/Real_...,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,2017,Poolians.com,Real Pool 3D - Poolians,Real Pool 3D - Poolians,Poolians.com
3,"[Action, Adventure, Casual]",http://store.steampowered.com/app/767400/2222/,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,2017,彼岸领域,弹炸人2222,弹炸人2222,彼岸领域
4,NaN,http://store.steampowered.com/app/773570/Log_C...,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,Dato no disponible,Sin dato disponible,Log Challenge,Sin dato disponible,Sin dato disponible


- ***'geners'***

La columna 'genres' esta formada por una lista de los distintos géneros de los videojuegos. Se necesita hacer una columna con cada género

In [29]:
df_games = df_games.explode('genres')
df_games = df_games.dropna(subset=['genres'])
df_games.head()

,genres,url,tags,reviews_url,specs,price,early_access,id,release_anio,publisher,app_name,title,developer
0,Action,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Casual,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Indie,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Simulation,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro
0,Strategy,http://store.steampowered.com/app/761140/Lost_...,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Lost Summoner Kitty,Kotoshiro


- ***'price'***

Se necesita trabajar con esta columna, pero hay valores de texto en casos de promociones o que el item es gratis. Por lo que se decide reemplazar esos valores por 0. Por otra parte, esta columna tiene algunos valores nulos, pero como mas adelante será necesario hacer operaciones, se adopta imputar como 0 los valores nulos.

In [30]:
df_games['price'] = df_games['price'].apply(herramientas.convierte_a_flotante)
df_games['price'].dtype

dtype('float64')

Se eliminan las columnas que no necesitamos, en este caso serian las columnas: 'publisher', 'url', 'app_name', 'specs', 'early_access', 'tags'. Luego visualizamos para verificar los cambios y guardamos en df_reviews

In [31]:
df_games_clean = df_games.drop('url', axis=1)
df_games_clean = df_games_clean.drop('specs', axis=1)
df_games_clean = df_games_clean.drop('tags', axis=1)
df_games_clean = df_games_clean.drop('reviews_url', axis=1)
df_games_clean = df_games_clean.drop('app_name', axis=1)

df_games = df_games_clean
df_games

,genres,price,early_access,id,release_anio,publisher,title,developer
0,Action,4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Kotoshiro
0,Casual,4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Kotoshiro
0,Indie,4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Kotoshiro
0,Simulation,4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Kotoshiro
0,Strategy,4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Kotoshiro
...,...,...,...,...,...,...,...,...
32132,Indie,1.99,False,610660,2018,Laush Studio,Russian Roads,Laush Dmitriy Sergeevich
32132,Racing,1.99,False,610660,2018,Laush Studio,Russian Roads,Laush Dmitriy Sergeevich
32132,Simulation,1.99,False,610660,2018,Laush Studio,Russian Roads,Laush Dmitriy Sergeevich
32133,Casual,4.99,False,658870,2017,SIXNAILS,EXIT 2 - Directions,"xropi,stev3ns"


In [32]:
herramientas.verifica_tipo_y_nulos(df_games)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,genres,[<class 'str'>],100.0,0.0,0
1,price,[<class 'float'>],100.0,0.0,0
2,early_access,[<class 'bool'>],100.0,0.0,0
3,id,[<class 'str'>],100.0,0.0,0
4,release_anio,[<class 'str'>],100.0,0.0,0
5,publisher,[<class 'str'>],100.0,0.0,0
6,title,[<class 'str'>],100.0,0.0,0
7,developer,[<class 'str'>],100.0,0.0,0


In [33]:
df_games

,genres,price,early_access,id,release_anio,publisher,title,developer
0,Action,4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Kotoshiro
0,Casual,4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Kotoshiro
0,Indie,4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Kotoshiro
0,Simulation,4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Kotoshiro
0,Strategy,4.99,False,761140,2018,Kotoshiro,Lost Summoner Kitty,Kotoshiro
...,...,...,...,...,...,...,...,...
32132,Indie,1.99,False,610660,2018,Laush Studio,Russian Roads,Laush Dmitriy Sergeevich
32132,Racing,1.99,False,610660,2018,Laush Studio,Russian Roads,Laush Dmitriy Sergeevich
32132,Simulation,1.99,False,610660,2018,Laush Studio,Russian Roads,Laush Dmitriy Sergeevich
32133,Casual,4.99,False,658870,2017,SIXNAILS,EXIT 2 - Directions,"xropi,stev3ns"


#### Carga del datasets: output_steam_games

Se guarda el conjunto de datos transformado como steam_games_limpio

In [34]:
df_games_limpio = 'data/steam_games_limpio.csv'
df_games.to_csv(df_games_limpio, index=False, encoding='utf-8')
print(f'Se guardó el archivo {df_games_limpio}')

Se guardó el archivo data/steam_games_limpio.csv
